In [50]:
#!pip install pandas polars bs4 requests 

import pandas as pd
import polars as pl
import bs4
import requests
import re
import os

In [51]:
df_postos = pl.read_parquet('./staging/pmqc_processed/postos.parquet')
df_postos

df_pmqc = pl.read_parquet('./staging/fuel_price_processed/precos.parquet')
df_pmqc = df_pmqc.select(pl.col('cnpj'), pl.col('nome_fantasia')).rename({'cnpj': 'CnpjPosto'})
df_pmqc


CnpjPosto,nome_fantasia
str,str
"""05849850000182…","""POSTO PAPALÉGU…"
"""05849850000182…","""POSTO PAPALÉGU…"
"""05849850000182…","""POSTO PAPALÉGU…"
"""05849850000182…","""POSTO PAPALÉGU…"
"""05849850000182…","""POSTO PAPALÉGU…"
"""02773620000199…","""""POSTO PIO XII…"
"""02773620000199…","""""POSTO PIO XII…"
"""04142631000104…","""AUTO POSTO LES…"
"""04142631000104…","""AUTO POSTO LES…"


In [52]:
df_precos = df_pmqc.join(df_postos, on='CnpjPosto', how='left')
df_precos = df_precos.rename({'RazaoSocialPosto': 'RSP1'})
df_precos = df_precos.rename({'nome_fantasia': 'RazaoSocialPosto'})
# when RazaoSocialPosto is null, then RSP1
df_precos = df_precos.with_columns(
    pl.when(pl.col('RazaoSocialPosto').is_null() | pl.col('RazaoSocialPosto').str.contains('nan'))
    .then(pl.col('RSP1'))
    .otherwise(pl.col('RazaoSocialPosto'))
    .alias('RazaoSocialPosto')
)
df_precos = df_precos.drop('RSP1')
df_precos = df_precos.unique(subset=['CnpjPosto'])
df_precos = df_precos.filter(pl.col('Latitude').is_not_null())
df_precos

CnpjPosto,RazaoSocialPosto,Distribuidora,DataColeta,Endereço,Latitude,Longitude,Bairro,Município,Uf,CnpjMatriz
str,str,str,date,str,f64,f64,str,str,str,i32
"""02756935000128…","""GAIVOTA""","""BANDEIRA BRANC…",2023-12-09,"""AVENIDA COTOVI…",-23.607297,-46.670765,"""MOEMA""","""SAO PAULO""","""SP""",2756935
"""00382609000172…","""POSTO TROPICAL…","""BANDEIRA BRANC…",2023-06-07,"""AVENIDA PRESID…",-3.828891,-38.523867,"""MONDUBIM""","""FORTALEZA""","""CE""",382609
"""11372084000453…","""POSTO OPCAO""","""IPIRANGA PRODU…",2020-12-14,"""RUA DEPUTADO B…",-7.115406,-34.873851,"""TAMBIA""","""JOAO PESSOA""","""PB""",11372084
"""36472132000163…","""POSTO COCK PIT…","""BANDEIRA BRANC…",2023-01-05,"""RUA DOS EXPEDI…",-22.810119,-43.411058,"""CENTRO""","""NILOPOLIS""","""RJ""",36472132
"""10583881000174…","""POSTO MANDINHO…","""BANDEIRA BRANC…",2021-01-20,"""AVENIDA PEDRO …",-15.795699,-43.327182,"""VEREDAS""","""JANAUBA""","""MG""",10583881
"""09527517000108…","""POSTO BARRA NO…","""IPIRANGA PRODU…",2022-03-01,"""AVENIDA BRASIL…",-26.97647,-48.637302,"""CENTRO""","""BALNEARIO CAMB…","""SC""",9527517
"""89470462002472…","""REDE ENERGIA""","""BANDEIRA BRANC…",2022-10-13,"""AVENIDA GETULI…",-29.88575,-51.17978,"""CENTRO""","""CANOAS""","""RS""",89470462
"""04596456000460…","""ANDREAZZA""","""PETROBRAS DIST…",2020-02-26,"""RUA ANGELO AGO…",-29.143636,-51.186022,"""PIONEIRO""","""CAXIAS DO SUL""","""RS""",4596456
"""13986632000118…","""MENDES & FILHO…","""IPIRANGA PRODU…",2019-01-18,"""ROD BR 153 KM …",-18.041639,-49.267626,"""POVOADO SERRIN…","""GOIATUBA""","""GO""",13986632


In [54]:
df_precos.write_json('./staging/pmqc_processed/postos.json', row_oriented=True)
df_precos.write_parquet('./staging/pmqc_processed/postos.parquet')